In [ ]:
%load_ext autoreload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import time
from scipy.optimize import minimize

import pyOptLE as optle

In [ ]:
%autoreload

In [ ]:
def overdamped_Langevin_ABMD(n, N, dt, beta = 1, D = .1, k = 10., nablaV = optle.nablaV_harm):
    
    np.random.seed(4)
    x = np.zeros([n, N])
    f = np.zeros([n, N])

    x0 = -5
    x[:,0] = np.zeros(n) + x0
    
    xmax = np.copy(x[:,0])
    xstop = np.zeros(n) + 4.

    for t in range(1,N):
        Fbiased = - nablaV(x[:,t-1]) + f[:,t-1]
        x[:,t] = x[:,t-1] + beta * D * Fbiased * dt + (
            np.random.normal(0, np.sqrt(2.*D*dt), size=n))
        np.maximum(xmax, x[:,t], out=xmax)
        np.minimum(xmax, xstop, out=xmax)
        f[:,t] = (k * (xmax - x[:,t]))
    
    return x, f


In [ ]:
generate = True
n = 1000  # number of synthetic trajectories
N = 500   # n. time steps per trajectory

do_undebiased = True

# User needs to provide dt!
dt = 0.02
T = 100

# data from "real" simulation
filenames = ['data_10ala/decaa_abf_500traj', 'data_10ala/decaa_abmd_500traj',
             'data_10ala/decaa_nobias_500traj', 'data_10ala/decaa_smd_500traj']

auto_bounds = True # If not, specify grid boundaries manually
# qmin_user = 11
# qmax_user = 34

n_knots = 30

In [ ]:
#D = 0.5
D = 1
R = 0.00198720425864 # kcal/mol
RT = R * T
beta = 1 / RT
k = 10
#     V = optle.V_harm
#     nablaV = optle.nablaV_harm
V = optle.V_dblwell
nablaV = optle.nablaV_dblwell

q, f = overdamped_Langevin_ABMD(n, N, dt, beta, D, k, nablaV)

#q, f = optle.overdamped_Langevin(n, N, dt, beta, D, nablaV), None


In [ ]:
plot_samples = not generate
plot_traj = True
n_plots = 10

if plot_samples:
    plt.plot([qi.shape[0] for qi in q])
    plt.xlabel('Trajectory index')
    plt.ylabel('N. of pts per trajectory')

if plot_traj:
    plt.figure()
    stride = len(q) // n_plots
    if stride == 0:
        stride = 1
    for qi, fi in zip(q[::stride], f[::stride]):
        plt.plot(qi)
        #plt.plot(fi)
    plt.xlabel('t')
    plt.ylabel('q')

In [ ]:
%matplotlib notebook

xmin = -5
xmax = 5
nbx = 100
x = np.linspace(xmin, xmax, nbx)

fig = plt.figure()
line, = plt.plot([], []) 
plt.xlim(xmin, xmax)
plt.ylim(np.min(V(x)), np.max(V(x)))


pot_curve, = plt.plot(x, V(x), label='V(x)', color='blue', linewidth=1.5)
bias_curve, = plt.plot([], [], '-', color='green', label='Vbiased(x)', linewidth=1.5)
max_line, = plt.plot([], [], ':', color='orange', label='max level', linewidth=1.5)
# force_curve, = plt.plot([], [])
pos, = plt.plot([], [], 'o', color='red')


bias_npoints = 100
bias_width = 4

itraj = 0

qmax = q[itraj][0]
qstop = 4.
Vmax = 10 # above top of graph

def animate(t):
    global qmax
    qt = q[itraj][t]
    if qt > qmax and qt <= qstop:
        qmax = qt
    pos.set_data((qt,), (V(qt)+0.5*k*np.square(qt-qmax),))
    bias_pts = np.linspace(qmax-bias_width, qmax, bias_npoints)
    bias_curve.set_data(bias_pts, V(bias_pts)+0.5*k*np.square(bias_pts-qmax))

#     Vb = V(qt) + 0.5*k*np.square(qt-qmax)
    max_line.set_data([qmax, qmax], [V(qmax), Vmax])

#     ft = f[itraj][t]
#     dVdq = (nablaV(qt) - ft) * force_width
#     force_curve.set_data([qt-force_width, qt + force_width],
#                          [Vb -dVdq, Vb + dVdq])
    return
    
ani = animation.FuncAnimation(fig, animate, frames=N,
                              interval=33, blit=True, repeat=False)

# animate(100)
plt.xlabel('x')
plt.ylabel('Potential energy')
plt.legend()


# writergif = animation.PillowWriter(fps=30) 
# ani.save('ABMD2.gif', writer=writergif)

plt.show()


In [ ]:
# Create grid and initial parameter set

epsilon = 1e-10
qmin_data = min([np.min(qi) for qi in q]) - epsilon
qmax_data = max([np.max(qi) for qi in q]) + epsilon

if auto_bounds:
    # Add margin for floating-point precision
    qmin = qmin_data
    qmax = qmax_data
else:
    assert qmin_user <= qmin_data, '%f > %f' % (qmin_user, qmin_data)
    assert qmax_user >= qmax_data, '%f < %f' % (qmax_user, qmax_data)
    qmin = qmin_user
    qmax = qmax_user


knots = np.linspace(qmin, qmax, n_knots)

# Initial guess for the parameters: zero free energy gradient, constant D = 1
initial_params = np.concatenate((np.zeros(n_knots), np.zeros(n_knots) + 1.))

In [ ]:
# Optimize with debiasing

result, hist = optle.optimize_model(initial_params, knots, q, f, dt, T = T)

In [ ]:
# Re-optimize without debiasing

if do_undebiased:
    result_nb, hist_nb = optle.optimize_model(initial_params, knots, q, f = None, dt = dt, T = T)


In [ ]:
# Smoother interpolation for plotting
interp_factor = 10

# Fencepost theorem aplied forward and then backward
n_discr_points = interp_factor * (len(knots) - 1) + 1
x = np.linspace(qmin, qmax, n_discr_points)

optimized_params = result.x
optimized_G = optimized_params[:n_knots]
optimized_logD = optimized_params[n_knots:]
# Integral of linear interpolation
predicted = optle.piecewise_linear_int(x, knots, optimized_G)

if do_undebiased:
    optimized_params_nb = result_nb.x
    optimized_G_nb = optimized_params_nb[:n_knots]
    optimized_logD_nb = optimized_params_nb[n_knots:]
    predicted_nb = optle.piecewise_linear_int(x, knots, optimized_G_nb)


# Plot the original data and the optimized spline
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure()

counts, bins = np.histogram(np.concatenate(q), bins=40)
# FE = -RT * np.log(counts)
# FE -= np.min(FE)
# plt.stairs(FE, bins, label='FE from histogram')
# R = 0.00198720425864 # kcal/mol
# RT = R * T
# plt.stairs(-RT*(np.log(counts)-np.max(np.log(counts))), bins, label='\"Free energy\" from histogram')

plt.plot(x, predicted, label='Optimized free energy (debiased)', marker='.', markevery=interp_factor)
if do_undebiased:
    plt.plot(x, predicted_nb, label='Optimized free energy (not debiased)', marker='.', markevery=interp_factor)

plt.xlabel('colvar q')
plt.ylabel('Free energy')

if generate:
    plt.plot(x, V(x) - np.min(V(x)), label='True potential')

# Ad hoc bc we know this is a double well
# plt.plot(x, optle.V_dblwell(x) - np.min(optle.V_dblwell(x)), label='Ideal double well')
plt.legend()


plot_gradient = True
if plot_gradient:
    plt.figure()
    plt.plot(knots, optimized_G, label='Optimized gradient', marker='.')
    if do_undebiased:
        plt.plot(knots, optimized_G_nb, label='Optimized gradient (not debiased)', marker='.')
    if generate:
        plt.plot(x, nablaV(x), label='True gradient')
        print("Gradient RMSD =", np.sqrt(np.mean(np.square(optimized_G[3:-3] - nablaV(knots[3:-3])))))
    plt.xlabel('colvar q')
    plt.ylabel('Free energy gradient')
    plt.legend()


plt.figure()
plt.plot(x, np.exp(np.interp(x, knots, optimized_logD)), label='Optimized D (debiased)',
         marker='.', markevery=interp_factor)
if do_undebiased:
    plt.plot(x, np.exp(np.interp(x, knots, optimized_logD_nb)), label='Optimized D (not debiased)',
             marker='.', markevery=interp_factor)

if generate:
    plt.plot([qmin, qmax], [D, D], label='True D')
plt.xlabel('colvar q')
plt.ylabel('Diffusion coefficient')
plt.legend()